# Calculate Area of wetlands

***
## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load Packages

In [1]:
import datacube
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rioxarray as rio
import xarray as xr
from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog
from datacube.utils.masking import mask_invalid_data
from deafrica_tools.classification import sklearn_flatten, sklearn_unflatten
from deafrica_tools.plotting import display_map, plot_lulc
from matplotlib import colors as mcolors
from matplotlib.colors import ListedColormap
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

### Analysis Parameters

* `pred_tif`: The path and name of the prediction GeoTIFF output in the previous notebook.
* `ouput_dir`: A folder location to store the classified GeoTIFFs.

In [2]:
class_dict = {
    "Cropland": 1,
    "Dense Forest": 2,
    "Open Forest": 3,
    "Open Grassland": 4,
    "Open Water": 5,
    "Otherland": 6,
    "Settlements": 7,
    "Vegetated Wetland": 8,
    "Wooded Grassland": 9,
}

In [3]:
years = ["1987", "2000","2007", "2017"]

output_dir = "results"

In [5]:
# Get the area of each land cover class in the prediction images.
results = {}
for year in years:
    pred_tif = f"{output_dir}/prediction_{year}.tif"
    # Load the pixel-based prediction raster.
    pred = rio.open_rasterio(pred_tif).squeeze()
    # Get the resolution i.e. actual ground distance represented by the length of a single pixel in meters.
    pixel_length = pred.geobox.resolution[1]
    # Conversion from metres to kilometres.
    m_per_km = 1000
    # Get the actual ground area represented by each pixel.
    area_per_pixel = (pixel_length / m_per_km) ** 2
    # Use the numpy np.unique function to return the pixel count for each land cover class.
    counts = np.unique(pred, return_counts=True)
    # Calculate the area of each land cover class.
    areas = np.array(counts[1] * area_per_pixel)
    results[year] = dict(zip(class_dict.keys(), areas))

    
 # Convert the results dictionary into a pandas DataFrame.
results_df = pd.DataFrame.from_dict(results)

results_df

,1987,2000,2007,2017
Cropland,7272.8748,14925.3948,18026.0226,26932.6932
Dense Forest,4312.8090,4163.9661,4569.2964,2740.6432
Open Forest,3207.7440,2836.6947,4487.6475,3836.8288
Open Grassland,6288.7797,4969.8783,4926.9240,4942.7872
Open Water,4448.8836,3618.5364,3524.3514,3475.5656
Otherland,30.7908,0.4320,2.0007,12.1496
Settlements,0.0198,1.9836,14.9085,125.2700
Vegetated Wetland,1388.4579,2620.7181,2278.8108,1989.1968
Wooded Grassland,22279.4595,16092.2151,11399.8572,5170.2040
